In [ ]:
"""
algorithms :

    SAE training :

dataset = base model dataset
activations = truncated_model(dataset)
SAE fit activations

    probe training :

dataset = string + label
activations = truncated_model(dataset)
    -> stop_at_layer (from transformer lens)
optional : activations = SAE(activations)
activations = trivial_normalisation(activations)

dataset = activations + label
loss = CCS or other
probe = probe
train probe

test generalisation
"""

In [9]:
import transformer_lens as tl

from probes import LinearProbe

In [14]:
gpt2small = tl.HookedTransformer.from_pretrained("gpt2")
d_resid = gpt2small.cfg.d_model
probe = LinearProbe(d_resid, 1)

Loaded pretrained model gpt2 into HookedTransformer


In [2]:
tl.loading_from_pretrained.OFFICIAL_MODEL_NAMES

['gpt2',
 'gpt2-medium',
 'gpt2-large',
 'gpt2-xl',
 'distilgpt2',
 'facebook/opt-125m',
 'facebook/opt-1.3b',
 'facebook/opt-2.7b',
 'facebook/opt-6.7b',
 'facebook/opt-13b',
 'facebook/opt-30b',
 'facebook/opt-66b',
 'EleutherAI/gpt-neo-125M',
 'EleutherAI/gpt-neo-1.3B',
 'EleutherAI/gpt-neo-2.7B',
 'EleutherAI/gpt-j-6B',
 'EleutherAI/gpt-neox-20b',
 'stanford-crfm/alias-gpt2-small-x21',
 'stanford-crfm/battlestar-gpt2-small-x49',
 'stanford-crfm/caprica-gpt2-small-x81',
 'stanford-crfm/darkmatter-gpt2-small-x343',
 'stanford-crfm/expanse-gpt2-small-x777',
 'stanford-crfm/arwen-gpt2-medium-x21',
 'stanford-crfm/beren-gpt2-medium-x49',
 'stanford-crfm/celebrimbor-gpt2-medium-x81',
 'stanford-crfm/durin-gpt2-medium-x343',
 'stanford-crfm/eowyn-gpt2-medium-x777',
 'EleutherAI/pythia-14m',
 'EleutherAI/pythia-31m',
 'EleutherAI/pythia-70m',
 'EleutherAI/pythia-160m',
 'EleutherAI/pythia-410m',
 'EleutherAI/pythia-1b',
 'EleutherAI/pythia-1.4b',
 'EleutherAI/pythia-2.8b',
 'EleutherAI/pyt